In [ ]:
import pandas as pd
import numpy as np
import infostop

In [ ]:
data = pd.read_csv('data_one_month.csv')
data = data.drop('Unnamed: 0', axis=1)
data

In [ ]:
# create model
model = infostop.Infostop()

In [ ]:
for key, group in data.groupby('user'):
    try:
        print(key)
        group = group.sort_values(by = 'timestamp', ascending = True)
        gps_data = group[['lat', 'lon', 'timestamp']].to_numpy()
        labels = model.fit_predict(gps_data)
        labels_list = labels.tolist()
        group['labels'] = labels_list 
        group = pd.DataFrame(infostop.postprocess.compute_intervals(group[['lat','lon','timestamp']].values,labels),
                columns = ['label','start','end','lat','lon'])
        stops_user = group[group['label'] != -1]
        stops_user = stops_user.reset_index(drop=True)
        stops_user['user'] = key
        final_stops = final_stops.append(stops_user)
    except:
        pass

In [ ]:
final_stops.to_csv('stops.csv', index=False)